## A Primer on Data Frames

So far we have seen data organized in various ways. We have seen data hiding on web pages, embedded in various HTML tags and attributes. We have been fed data in JSON format via various APIs -- JSON which can be represented as built-in Python objects like dictionaries and lists and numbers and character strings. 

Despite these formats, it is often the case that data arrange themselves naturally in a table. In some sense, the sole of the statistical enterprise depends on tabular data. Each row of a table represents a unit of observation (a city, a person), and the columns represent measurements on these observations (tweet count, a time of day, a hashtag). 

The CSV format is all about tables. Every spreadsheet you've ever worked with is a table. This format is so common that it was only a matter of time before someone contributed a high-level abstraction, an object, that represents a table in Python. [Enter Pandas](https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/).
<br><br>
<img src="https://qzprod.files.wordpress.com/2017/11/programmer.jpg?quality=80&strip=all&w=3200" style="width: 85%; border: #000000 1px outset;">
<br><br>
There is extensive [documentation on Pandas](https://pandas.pydata.org/pandas-docs/stable/) and this document is meant to take you through some of the basics. Specifically, the kinds of computations that we do day-after-day. Our test case will be (and this is the VERY last time we will bring it up) the hashtags we studied last week. While we already have data about the tweets including each of \#schumershutdown and \#trumpshutdown, we have found a source of historical trends data. It is given to us by `itrended.com` -- anyone wanting an account to do other kinds of searches can obtain one... just write me.
<br><br>
<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/trit.jpg" style="width: 65%; border: #000000 1px outset;">
<br><br>
Let's begin! First, just to make our ourput tidy, we are going to set an option to control how our tables are displayed. We can use the command `set_option` to fix the number of rows or columns are shown, or maybe how many characters are displayed in each table cell before the data are truncated. Here we want pretty short table output just to keep things readable.

In [ ]:
from pandas import set_option
set_option('display.max_rows', 20)

**1. Making a Data Frame**

The simplest way to create a Data Frame is to read a CSV file. There is a special function called `read_csv` in the Pandas module that does this for you. The CSV file can be on your computer or on the web somewhere. Here we read a CSV from our Github repository. 

In [ ]:
from pandas import read_csv
trump = read_csv("https://raw.githubusercontent.com/computationaljournalism/columbia2018/master/data/trumptrend.csv")
print(type(trump))

The CSV file came from `iTrended.com` and was the result of a search for \#trumpshutdown. The function `read_csv` returns an object of type `DataFrame`. Remember software objects contain data, and offer actions that you can execute on them.

For example, important pieces of data for a table would be its row and column count. *How much data are we talking about?* We access these figures with the `shape` attribute of the DataFrame object using the (now familiar) "dot" notation....

In [ ]:
trump.shape

... which tells us we have 43K rows and 5 columns. As an example of an operation you can execute on a DataFrame, let's look at the top and bottom of the table or spreadsheet -- its `head()` and `tail()`. 

In [ ]:
trump.head()

In [ ]:
trump.tail()

The columns are the `TREND` name, the date and time (in UTC) that the trend was recorded, the `WOEID` or Where on Earth ID for the place, the place's name, and the rank of the trend at that time in that place. So 45 minutes after midnight UTC on January 19, \#trumpshutdown was ranked 47th among the trending topics in Portland.

**2. Sorting**

Another thing we might want to do is reorder our rows, sorting them according to one or more variables. Here we sort by `TREND_POSITION`. What happens?

In [ ]:
trump.sort_values("TREND_POSITION")

The sorting has ordered the rows of `trump` so that the times and places where this hashtag was at the top of the list (rank 1) come first, then the 2nd places and so on. You can sort by more than one variable by simply passing a list of names of columns to sort by. Here we sort by `TREND_POSITION` first, and then among all of those with the same `TREND_POSITION` we order by time.

In [ ]:
trump.sort_values(["TREND_POSITION","DATETIME_UTC"])

And we see that Phoenix was the first to put this hashtag at the top of its trending list, at about 11:20 UTC on January 19.

Sometimes, the order of the data matter. We might want to process it in a way that depends on knowing that each row is a step later in time, say. So, in this case we might simply replace our original data set `trump`. We can do this by adding an "argument" to the `sort_values` command called `inplace`. That tells Pandas to sort the data and replace the unsorted `trump` with a new, sorted version.

In [ ]:
trump.sort_values("DATETIME_UTC",inplace=True)

In [ ]:
trump.head()

In [ ]:
trump.tail()

And now we see that the date ranges run from January 19 through the 24th. Let's read in the counterpart to \#trumpshutdown, #schumershutdown.

In [ ]:
from pandas import read_csv
schumer = read_csv("https://raw.githubusercontent.com/computationaljournalism/columbia2018/master/data/schumertrend.csv")
print(type(schumer))

Again, we get a DataFrame and we'll sort the table by time.

In [ ]:
schumer.sort_values("DATETIME_UTC",inplace=True)

In [ ]:
schumer.head()

In [ ]:
schumer.tail()

**3. Extracting a single column of data from the table**

So far we have looked at whole-table operations. Often, we are interested in the data from one or more individual columns. We extract columns using square brackets. Just like we did for lists and dictionaries, [&nbsp;&nbsp;]'s mean subsetting, pulling out data. 

For a Pandas object, we can pull out a single column by referring to its name in square brackets.

In [ ]:
schumer["LOCATION_NAME"]

In [ ]:
print(type(schumer["LOCATION_NAME"]))

When we pull a single column of data from a Data Frame, its type is a `Series`. You can think of this as a fancy list. I mean a column of data has a first entry, a second entry and so on, as you work your way down the table. (Why Virginia beach is among the last places for this hashtag to trend is beyond me.) It is a fancy list because you can do other things to it, operations that are common "next steps" in an analysis.

For example, once we pull out a column, we might want to summarize it. How many times was \#schumershutdown trending in different places around the globe?

In [ ]:
schumer["LOCATION_NAME"].value_counts()

Oh Bournemouth and Stoke-on-Trent, blissfully removed from our government's shenannigans. For a categorical variable (like location names), counting the number of each category is a good summary. For numerical variables, like rank, we might consider...

In [ ]:
schumer["TREND_POSITION"].max()

In [ ]:
schumer["TREND_POSITION"].min()

... or a more complete numeric summary (the 6-number summary, more or less, if you've had statistics).

In [ ]:
schumer["TREND_POSITION"].describe()

Just to make things complicated, for simple functions like `max` and `min`, there are also simple built-in functions that can take the columns as arguments. So instead of the two expressions above we can also use the following.

In [ ]:
max(schumer["TREND_POSITION"])

In [ ]:
min(schumer["TREND_POSITION"])

Numerical calculations are easy with `Series` objects as they allow you to do arithmetic along entire columns. For example, we here's how we would multiply each rank in the table by 100.

In [ ]:
schumer["TREND_POSITION"] * 100

The beauty here is that we don't have to loop over elements in a column we can perform so-called "vectorized" operations. One expression, one operator, works across the entire column. 

**4. Working with booleans**

In addition to numeric operators, there are also logical or boolean operators that we have mentioned before. 

>Equals: ==
<br>Not equals: !=
<br>Greater than, less than: > or <
<br>Greater than or equal to: >=
<br>Less than or equal to: <=

These also apply down an entire column, making a series of comparisons. Here we return a `Series` that holds `True` if the `TREND_POSITION` is 1 and `False` otherwise.

In [ ]:
schumer["TREND_POSITION"] == 1

We can now `sum()` or `.sum()` (remember the two versions of `max` and `min`) this column to tell us how many `True`'s we have. Each `True` turns to a 1 when you include it in numerical calculations like a sum. 

In [ ]:
sum(schumer["TREND_POSITION"] == 1)

And so we have 199 places and times when this hashtag was ranked first, and we have...

In [ ]:
sum(schumer["LOCATION_NAME"]=="Washington")

531 times that this hashtag was trending in Washington DC.
<br><br>
<img src="https://github.com/computationaljournalism/columbia2018/raw/master/images/fl.jpg" style="width: 65%; border: #000000 1px outset;">
<br><br>
Ah, but you are not irresistably led to ask, how many tmes was this hashtag number 1 in Washington? Come on, I know you are! We can join boolean expressions together with `&` for "and" and `|` for "or". In the former case, we have a `True` if both expressions are true, and in the latter case only one expression has t be true.

Here is what an `&` looks like. Again, we get back a `Series` of boolean data, one element for each row in the `schumer` table. 

In [ ]:
(schumer["TREND_POSITION"]==1) & (schumer["LOCATION_NAME"]=="Washington")

Summing this column tells us how much time this hashtag was at the top of the list in Washington. 

In [ ]:
sum((schumer["TREND_POSITION"]==1) & (schumer["LOCATION_NAME"]=="Washington"))

That's 44 5-minute periods, or 3 hours and 40 minutes. What about Trump?

In [ ]:
sum((trump["TREND_POSITION"]==1) & (trump["LOCATION_NAME"]=="Washington"))

In [ ]:
print("Schumer spent",44*5/60,"hours at #1 in Washington")
print("Trump spent",195*5/60,"hours at #1 in Washington")

Just as an aside, you get a different picture if you look at the top 10. So now, how often was each among the top 10 trends in Washington?

In [ ]:
sum((schumer["TREND_POSITION"]<=10) & (schumer["LOCATION_NAME"]=="Washington"))

In [ ]:
sum((trump["TREND_POSITION"]<=10) & (trump["LOCATION_NAME"]=="Washington"))

**5. Selecting data using labels (column headings)**

So far we have seen that we can use the square brackets to extract single columns of a DataFrame. This returned a `Series`, a kind of fancy list, with each element representing the data from a row in the column. Instead of passing a single string or column name, we can return a smaller DataFrame by specifying a list of column names. Here we look at just the `DATETIME_UTC`, `LOCATION_NAME` and the `TREND_POSITION`.  

I've added spaces so you see that inside the square brackets is a list of column names.

In [ ]:
trump[ ["DATETIME_UTC","LOCATION_NAME","TREND_POSITION"] ]

**6. Selecting rows by slices**

One of the idiosyncratic features of Pandas is that the square brackets are a bit overloaded notationally. You can use them to pull out a column or to pull out a smaller DataFrame. If you pass a "slice" of numeric indices, you can pull out select rows. 

So, to pull out the 30th through the 40th row of the `schumer` DataFrame, we would do the following. (Remember we start counting at 0)

In [ ]:
schumer[29:40]

This notation should be familiar. It mimics what we did with both lists and strings. Wes, Pandas' designer, wisely used a notational convention that Python users would be familiar with. So while it might seem a little strange to have the square brackets for a DataFrame do two things, well, it represents the fact that we do these two things **a lot**. 

For the price of a little notational overloading, we get the operations we want, cleanly. We will see a more elaborate way to subset that is less pretty but maybe more proper in some sense for those sticklers in the crowd.

**7. Subsetting rows based on a logical mask**

While it is often the case that we know what number rows we want to extract, we are more commonly able to specify a set of conditions that define the rows we want to keep. Say, all the ranks that occurred in Washington DC...

In [ ]:
trump["LOCATION_NAME"]=="Washington"

... or Tallahassee.

In [ ]:
trump["LOCATION_NAME"]=="Tallahassee"

When we supply the square brackets of a DataFrame with a boolean `Series`, Pandas will return just those rows associated with `True`, or satisfying our condition. Here is the case for \#trumpshutdown in Washington. I have again expanded the space around the outer square brackets to highlight what's on the inside.

In [ ]:
trump_wash = trump[ trump["LOCATION_NAME"]=="Washington" ]
trump_wash.head()

In [ ]:
trump_wash.shape

As we computed before, that's 255 entries, but here they are in a data frame.  We can then use this reduced data frame to make a plot, say, of when these ranking events occurred and what rank the hashtag achieved.

In [ ]:
# the good old plotly plot

from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go 

# sign in
sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

# create a data frame from Washington
trump_wash = trump[trump["LOCATION_NAME"]=="Washington"]

# create a plot with time on the x and rank on the y.
# the "layout" specifies that we reverse the y-axis so that 1 is at the top and 50
# at the bottom.

myplot_parts = [go.Scatter(x=trump_wash["DATETIME_UTC"],y=trump_wash["TREND_POSITION"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=400,yaxis={"autorange":"reversed"})


myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="washington-trump")

And here is the same thing, but for Schumer.

In [ ]:
schum_wash = schumer[schumer["LOCATION_NAME"]=="Washington"]

myplot_parts = [go.Scatter(x=schum_wash["DATETIME_UTC"],y=schum_wash["TREND_POSITION"],mode="markers")]
mylayout = go.Layout(autosize=False, width=800,height=400,yaxis={"autorange":"reversed"})
    
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="washington-schum")

And just for grins, we put the same two plots over the top of each other. Notice the list of `myplot_parts` has two elements, one for Schumer and one for Trump. Plotly will take care of the color.

In [ ]:
schum_wash = schumer[schumer["LOCATION_NAME"]=="Washington"]
trump_wash = trump[trump["LOCATION_NAME"]=="Washington"]

myplot_parts = [go.Scatter(x=schum_wash["DATETIME_UTC"],y=schum_wash["TREND_POSITION"],mode="markers",name="Schumer"),
                go.Scatter(x=trump_wash["DATETIME_UTC"],y=trump_wash["TREND_POSITION"],mode="markers",name="Trump")]
mylayout = go.Layout(autosize=False, width=800,height=400,yaxis={"autorange":"reversed"})
    
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="washington-schum")

**$\infty$ Your turn**

You can now practice subsetting on a similar data frame (the columns are all the same) but now we asked `itrended.com` for the history of \#ReleaseTheMemo. 

In [ ]:
memo = read_csv("https://github.com/computationaljournalism/columbia2018/raw/master/data/memo.csv")
memo.head()

Here is a simple plot just to see what happened in Washington, say.

memous = memo[memo["LOCATION_NAME"]=="Washington"]

myplot_parts = [go.Scatter(x=memous["DATETIME_UTC"],y=memous["TREND_POSITION"],mode="markers")]
mylayout = go.Layout(autosize=False,width=800,height=600,yaxis={"autorange":"reversed"})
    
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="memo")

Now, where to go? You might consider thinking about the analysis in [this Politico article](https://www.politico.com/magazine/story/2018/02/04/trump-twitter-russians-release-the-memo-216935). We can pull things from Twitter directly, make use of online services like `itrended.com` for historical data, and even consider the Hamilton 68 methodology. Why not implement this ourselves?